In [10]:
#!python -m pip install --upgrade pip
#!pip -q install --upgrade transformers
#!pip -q install --upgrade accelerate
#!pip -q install --upgrade torch
#!pip -q install --upgrade sentencepiece
#!pip install --upgrade protobuf==3.19.0
!pip install --upgrade tokenizers

  Using cached tokenizers-0.13.3-cp38-cp38-macosx_10_11_x86_64.whl (4.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2


In [2]:
!python -V

Python 3.8.8


In [9]:
#import shutil
#shutil.rmtree('llama/hf')
#!mkdir llama
#!mkdir 'llama/hf-7b'

In [1]:
!apt-get install gcc python3-dev python3-setuptools
!pip3 uninstall crcmod
!pip3 install --no-cache-dir -U crcmod

zsh:1: command not found: apt-get
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for crcmod: filename=crcmod-1.7-cp38-cp38-macosx_10_9_x86_64.whl size=22358 sha256=a3824f5e3a4bb5c039d2c5f4101fbfe1854e3cc52ea73a129b097a3dea32bd57
  Stored in directory: /private/var/folders/1t/v0bz2cw178lgvqvt64jr0fgm0000gn/T/pip-ephem-wheel-cache-sl9von9x/wheels/ca/5a/02/f3acf982a026f3319fb3e798a8dca2d48fafee7761788562e9
Successfully built crcmod


In [2]:
!gsutil -m cp -r gs://calibragpt/llama/7B llama/

If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Copying gs://calibragpt/llama/7B/checklist.chk...
Copying gs://calibragpt/llama/7B/consolidated.00.pth...
Copying gs://calibragpt/llama/7B/params.json...

Operation completed over 3 objects/12.6 GiB.                                     


In [3]:
!gsutil cp gs://calibragpt/llama/llama.sh llama
!gsutil cp gs://calibragpt/llama/tokenizer.model llama
!gsutil cp gs://calibragpt/llama/tokenizer_checklist.chk llama

Copying gs://calibragpt/llama/llama.sh...

Operation completed over 1 objects/1.9 KiB.                                      
Copying gs://calibragpt/llama/tokenizer.model...

Operation completed over 1 objects/488.0 KiB.                                    
Copying gs://calibragpt/llama/tokenizer_checklist.chk...

Operation completed over 1 objects/50.0 B.                                       


In [4]:
!ls llama

7B                      llama.sh                tokenizer_checklist.chk
hf-7b                   tokenizer.model


In [5]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 144570, done.
remote: Counting objects: 100% (814/814), done.
remote: Compressing objects: 100% (496/496), done.
remote: Total 144570 (delta 406), reused 590 (delta 266), pack-reused 143756
Receiving objects: 100% (144570/144570), 147.92 MiB | 19.45 MiB/s, done.
Resolving deltas: 100% (106944/106944), done.


In [7]:
!ls

LLaMA-weight-transfer.ipynb load_model.py
llama                       transformers
load_model.ipynb


In [11]:
!python transformers/src/transformers/models/llama/convert_llama_weights_to_hf.py \
    --input_dir llama --model_size 7B --output_dir llama/hf-7b

Fetching all parameters from the checkpoint at llama/7B.
Loading the checkpoint in a Llama model.
Loading checkpoint shards: 100%|████████████████| 33/33 [00:07<00:00,  4.47it/s]
Saving in the Transformers format.
Saving a LlamaTokenizerFast to llama/hf-7b.


In [9]:
!ls

LLaMA-weight-transfer.ipynb load_model.py
llama                       transformers
load_model.ipynb


In [13]:
from transformers import LlamaForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("llama/hf-7b")
model = LlamaForCausalLM.from_pretrained("llama/hf-7b")
model
# tokenizer

/Users/danielfurman/Desktop/Berkeley/Semester-4/Capstone/.venv_capstone/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [01:45<00:00, 52.84s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [12]:
#!gsutil -m cp -r llama/hf gs://calibragpt/llama

In [14]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

"Hey, are you conscious? Can you talk to me?\nI'm not sure if you're conscious, but I'm going"